## __Chapter 03: Untitled__
---
by Daniel Weller & Holger Mann

In [ ]:
data.packages <- c("bigmemory", "data.table", "rgdal", "rgeos")
lapply(data.packages, require, character.only=T); rm(data.packages)

stat.packages <- c("bivariate", "extRemes", "ks", "mev", "mvtnorm")
lapply(stat.packages, require, character.only=T); rm(stat.packages)

if(.Platform$OS.type!="windows"){
  paths <- c("/Volumes/Climate Rasters/", "/Volumes/Analysis Storage", "~/GitHub/Report/")} else {
    paths <- c("D:/Climate Rasters/", "E:/Analysis Storage/", "C:/Users/Cloud/GitHub/Report/")}
setwd(paths[3]); options(scipen=10); set.seed(12345); setDTthreads(16)
rasterOptions(progress="text", maxmemory=1e+09, chunksize=1e+08)

In [ ]:
rtn.years <- seq(10, 50, by=10); heat.rank <- c(0, 50, 100, 155, 225, 300, 400, 525, 700, 875)/1000
cold.count <- big.matrix(nrow=cells, ncol=length(rtn.years), init=NULL, separated=FALSE,
                         backingpath=".", backingfile="Cold_Count_Back", descriptorfile="Cold_Count_Desc")

for(i in 1:length(cold.rank)){
  for(j in cell.list){
    values <- fread(paste(paths[2], "Cold Wave Large/Cell ", j, ".csv",sep=""))
    values <- values[, Rise := round(Thresh, digits=2) - MaxDay][WaveID >= 1]
    values[, `:=`(Excess=sum(round(Rise, digits=2)), Length=max(DaysCount, na.rm=TRUE)), by=.(Year, WaveID)]
    values <- unique(values[, .(Year, WaveID, Excess, Length)])[Excess >= 0.03 & Length >= 3]
    for(j in 1:nrow(values)){values[j, Prob := cold.fun(values[j,3], values[j,4])]}
    values <- values[, .(Count=sum(Prob >= cold.rank[i], na.rm=TRUE)), by=.(Year)]
    try(model <- fevd(values$Count, threshold=0, type="PP", span=years))
    ifelse(exists("model"), cold.count[j,] <- return.level(model, rtn.years)[1:5], cold.count[j,] <- 0)
    cat(paste("Current Cell:", j, "in Category", i, "\n", seq="")); try(rm(model, values), silent=TRUE)
    if(j%in%seq(1, cells, by=1e+2)){invisible(gc())}}
  write.big.matrix(cold.count, paste("./Cold_Wave_Count_", i,".csv", sep="")); cold.count <- NA}


heat.count <- big.matrix(nrow=cells, ncol=length(rtn.years), init=NULL, separated=FALSE,
                         backingpath=".", backingfile="Heat_Count_Back", descriptorfile="Heat_Count_Desc")
for(i in 1:length(heat.rank)){
  for(j in cell.list){
    values <- fread(paste(paths[2], "Heat Wave Large/Cell ", j, ".csv",sep=""))
    values <- values[, Rise := MaxDay - round(Thresh, digits=2)][WaveID >= 1]
    values[, `:=`(Excess=sum(round(Rise, digits=2)), Length=max(DaysCount, na.rm=TRUE)), by=.(Year, WaveID)]
    values <- unique(values[, .(Year, WaveID, Excess, Length)])[Excess >= 0.03 & Length >= 3]
    for(k in 1:nrow(values)){values[k, Prob := heat.fun(values[k,3], values[k,4])]}
    values <- values[, .(Count=sum(Prob >= heat.rank[i], na.rm=TRUE)), by=.(Year)]
    try(model <- fevd(values$Count, threshold=0, type="PP", span=years), silent=TRUE)
    ifelse(exists("model"), heat.count[j,] <- return.level(model, rtn.years)[1:5], heat.count[j,] <- 0)
    cat(paste("Current Cell:", j, "in Category", i, "\n", seq="")); try(rm(model, values), silent=TRUE)
    if(j%in%seq(1, cells, by=1e+2)){invisible(gc())}}
  write.big.matrix(heat.count, paste("./Heat_Wave_Count_", i,".csv", sep="")); heat.count[,] <- NA}